In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os
sys.path.insert(0, os.path.abspath('..'))
from secret import AK_SK
keys = AK_SK('../qianfan.keys')
os.environ["QIANFAN_ACCESS_KEY"] = keys.get_ak()
os.environ["QIANFAN_SECRET_KEY"] = keys.get_sk()

In [23]:
from myqianfan import QianfanLLM, QianfanEmbedding
from myqianfan import model_spec

llm = QianfanLLM(model_spec=model_spec.ERNIE4_Turbo8K, temperature=0.5)

In [4]:
from atlassian import Confluence
from langchain import hub
from langchain_chroma import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import ConfluenceLoader
from langchain.schema import BaseOutputParser

In [ ]:
vectorstore = Chroma("qianfan", embedding_function=QianfanEmbedding(), persist_directory="vectorstore/qianfan.vec")

## 第一次初始化时运行

In [7]:
def get_children_pages_recursively(client, page_id: str):
    child_pages = client.get_page_child_by_type(page_id)
    for page in child_pages:
        yield page
        if "id" in page:
            yield from get_children_pages_recursively(client, page["id"])

from getpass import getpass
wiki_psw = getpass("wiki:")
# get all pages under some page
confluence = Confluence(
    url='https://wiki.fintopia.tech/',
    username='tongxinwen@fintopia.tech',
    password=wiki_psw)
child_pages = [p for p in get_children_pages_recursively(confluence, "74290517")]

loader = ConfluenceLoader(
    url="https://wiki.fintopia.tech/",
    page_ids=[p["id"] for p in child_pages],
    username="tongxinwen@fintopia.tech",
    api_key=wiki_psw,
    limit=10,
)
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=100)
splits = text_splitter.split_documents(documents)

batch_size = 16
doc_ids = []
for i in range(0, len(splits), batch_size):
    print(f"Adding documents {i} to {i+batch_size}")
    texts = [doc.page_content for doc in splits[i:i+batch_size]]
    metadatas = [doc.metadata for doc in splits[i:i+batch_size]]
    doc_ids += vectorstore.add_texts(texts, metadatas)

66

In [21]:
# vectorstore.search("数据库", search_type="similarity")

[Document(metadata={'id': '95160416', 'source': 'https://wiki.fintopia.tech/display/riskDev/2024H2', 'title': '2024H2', 'when': '2024-07-09T17:39:33.000+08:00'}, page_content='200ms KR3 模型推理框架国内外统一 【P2】 KR4 模型cache表优化，提高插入和查询效率，同时降低存储压力【P2】 建立HBase + Hive + PolarDB 3位一体的多级缓存 查询方面通过PolarDB + HBase提供高效查询，存储方面通过Hive提供长期存储功能，方面回溯各种问题 建立各级存储模块的过期策略，在PolarDB中存储原来的30%的数据，HBase存储未过期model的所有记录，hive存储最近3年的所有记录 O5 海外平台搭建 【P0】 以菲律宾、印尼为主 目标，训练全部迁移到ailab，推理迁移到新框架。'),
 Document(metadata={'id': '95160416', 'source': 'https://wiki.fintopia.tech/display/riskDev/2024H2', 'title': '2024H2', 'when': '2024-07-09T17:39:33.000+08:00'}, page_content='200ms KR3 模型推理框架国内外统一 【P2】 KR4 模型cache表优化，提高插入和查询效率，同时降低存储压力【P2】 建立HBase + Hive + PolarDB 3位一体的多级缓存 查询方面通过PolarDB + HBase提供高效查询，存储方面通过Hive提供长期存储功能，方面回溯各种问题 建立各级存储模块的过期策略，在PolarDB中存储原来的30%的数据，HBase存储未过期model的所有记录，hive存储最近3年的所有记录 O5 海外平台搭建 【P0】 以菲律宾、印尼为主 目标，训练全部迁移到ailab，推理迁移到新框架。'),
 Document(metadata={'id': '91856753', 'source': 'https://wiki.fintopia.tech/pages/vi

## 构建Rag chain

In [11]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
        [
            ("human", "你是一个解答用户问题的assistant，可以根据从语料库中召回的context文本回答问题。请尽量保证回答的内容都可以在context中找到根据，并务必保留==>source后面的url链接。以下是context文本：{context}，问题是：{question}"),
            ("assistant", "根据context文本，我认为答案是："),
        ]
    )

In [33]:
prompt.parse_raw(llm.conversation_history[-2]['content'])

ValidationError: 1 validation error for ChatPromptTemplate
__root__
  Expecting value: line 1 column 1 (char 0) [type=value_error.jsondecode, input_value='Human: 你是一个解...我认为答案是：', input_type=str]

In [24]:
retriever = vectorstore.as_retriever()

def format_docs(docs):
    return "\n\n".join(f"{d.page_content}\n==>source: {d.metadata['source']}" for d in docs) 

def retrieve_from_conversation_history(llm, retriever):
    question_history = [c['content'] for c in llm.get_conversation_history() if c['role'] == 'user']
    return vectorstore.search(conversation_history, search_type="similarity")

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [25]:
rag_chain.invoke("公共资源池和私有资源池的区别")

'公共资源池和私有资源池的区别主要在于付费方式、资源使用优先级、创建和启动实例的速度以及资源的归属。\n\n1. 付费方式：公共资源池是后付费的，根据小时数计费，不用不计费。而私有资源池是预付费的，即钱已经花出去了，不用就浪费了。==>source: [https://wiki.fintopia.tech/pages/viewpage.action?pageId=80187151](https://wiki.fintopia.tech/pages/viewpage.action?pageId=80187151)\n2. 资源使用优先级：当用户提交任务时，系统会优先选择私有资源池的资源，如果私有资源池没有资源了，才会选择公共资源池。==>source: [https://wiki.fintopia.tech/pages/viewpage.action?pageId=106844570](https://wiki.fintopia.tech/pages/viewpage.action?pageId=106844570)\n3. 创建和启动实例的速度：私有资源池的特点是创建和启动实例快，但如果用的人多了可能没资源。而公共资源池的特点是创建和启动实例比较慢。==>source: [https://wiki.fintopia.tech/pages/viewpage.action?pageId=80187151](https://wiki.fintopia.tech/pages/viewpage.action?pageId=80187151)\n4. 资源的归属：公共资源池属于阿里云的大池子，而私有资源池是我们自己买的一个小池子。==>source: [https://wiki.fintopia.tech/pages/viewpage.action?pageId=80187151](https://wiki.fintopia.tech/pages/viewpage.action?pageId=80187151)'

In [27]:
rag_chain.invoke("如何进行分布式LightGBM模型训练")

'进行分布式LightGBM模型训练，可以利用Ray框架来实现。Ray是一款支持LightGBM等多种模型框架的分布式训练机器学习框架。它的LightGBM分布式训练方法本身是利用了LightGBM自身的分布式训练功能，Ray只是为其提供了一套多机的分布式计算框架。关于LightGBM的分布式训练原理，可以参考其官方文档：https://lightgbm.readthedocs.io/en/latest/Features.html#optimization-in-distributed-learning。\n\n==>source: https://wiki.fintopia.tech/pages/viewpage.action?pageId=97186155'

In [28]:
rag_chain.invoke("需要具备哪些前提条件")

'根据提供的context文本，进行分布式LightGBM模型训练或相关操作需要具备以下前提条件：\n\n1. 需要有一个开关结合待替换模型id列表配置和服务名称的配置。这是在构建过程中判断是否需要使用复制的模型id来查询特征依赖进行构建的基础。\n==>source: https://wiki.fintopia.tech/pages/viewpage.action?pageId=84512081\n\n2. 在特定的环境（mirror或prod）中，需要满足特定的条件，如model-mirror环境中模型被复制了一个新modelInfo，或者在prod环境中useNewFeatureListSwitch开启且待替换模型id列表中包含当前模型id。\n==>source: https://wiki.fintopia.tech/pages/viewpage.action?pageId=84512081\n\n3. 对于文件填写要求，需要上传特定格式的txt文件，文件中每行代表一个特征，特征之间以\\t分隔，且文件从第二行开始读取。这是进行模型训练或部署时，对输入数据的基本要求。\n==>source: https://wiki.fintopia.tech/pages/viewpage.action?pageId=85647676'

In [30]:
llm.conversation_history[-2]

{'role': 'user',
 'content': 'Human: 你是一个解答用户问题的assistant，可以根据从语料库中召回的context文本回答问题。请尽量保证回答的内容都可以在context中找到根据，并务必保留==>source后面的url链接。以下是context文本：增加一个开关结合待替换模型id列表配置和服务名称,在构建时如果符合条件,则将去查当前模型复制出来的模型id,使用复制的模型id查询特征依赖进行构建 具体判断如下 if （model-mirror && 通过model_id查询到它复制了一个新modelInfo) || (useNewFeatureListSwitch &&\xa0toReplaceModelList.contains(modelId) )： // 查询复制的新model info，并用它构建元数据 else: // 跟原来一样 if 条件的第一部分是在mirror环境中生效，第二部分在prod环境生效。 配置 样例 说明\n==>source: https://wiki.fintopia.tech/pages/viewpage.action?pageId=84512081\n\n增加一个开关结合待替换模型id列表配置和服务名称,在构建时如果符合条件,则将去查当前模型复制出来的模型id,使用复制的模型id查询特征依赖进行构建 具体判断如下 if （model-mirror && 通过model_id查询到它复制了一个新modelInfo) || (useNewFeatureListSwitch &&\xa0toReplaceModelList.contains(modelId) )： // 查询复制的新model info，并用它构建元数据 else: // 跟原来一样 if 条件的第一部分是在mirror环境中生效，第二部分在prod环境生效。 配置 样例 说明\n==>source: https://wiki.fintopia.tech/pages/viewpage.action?pageId=84512081\n\n2、文件填写要求： （1）上传txt文件，一行一个特征，以\\t分隔。无需分箱分箱规则不填写 （2）文件从第二行开始读取 （3）示例：索引 特征名称 特征场景 默认值 是否有flag特征 分箱规